In [1]:
import pandas as pd

In [2]:
price_data = pd.read_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Dados_bolsa.xlsx', sheet_name= "Sheet1")
price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2000-01-05,0.520882,0.520882,0.520882,0.520882,0.297814,985,ABEV3.SA
1,2000-01-06,0.494478,0.494478,0.494478,0.494478,0.282718,227,ABEV3.SA
2,2000-01-07,0.494478,0.494478,0.494478,0.494478,0.282718,151,ABEV3.SA
3,2000-01-10,0.494478,0.494478,0.494478,0.494478,0.282718,1516,ABEV3.SA
4,2000-01-11,0.494478,0.494478,0.494478,0.494478,0.282718,3791,ABEV3.SA


In [5]:
price_data = price_data.rename(columns={'Ticker':'symbol', 'Date':'datetime','Close':'close','High':'high','Low':'low','Open':'open', 'Volume':'volume'})

price_data = price_data[['symbol','datetime','close','high','low','open','volume']]

# sort the values by symbol and then date
price_data.sort_values(by = ['symbol','datetime'], inplace = True)

# calculate the change in price
price_data['change_in_price'] = price_data['close'].diff()
price_data.head(10)

,symbol,datetime,close,high,low,open,volume,change_in_price
0,ABEV3.SA,2000-01-05,0.520882,0.520882,0.520882,0.520882,985,NaN
1,ABEV3.SA,2000-01-06,0.494478,0.494478,0.494478,0.494478,227,-0.026404
2,ABEV3.SA,2000-01-07,0.494478,0.494478,0.494478,0.494478,151,0.000000
3,ABEV3.SA,2000-01-10,0.494478,0.494478,0.494478,0.494478,1516,0.000000
4,ABEV3.SA,2000-01-11,0.494478,0.494478,0.494478,0.494478,3791,0.000000
5,ABEV3.SA,2000-01-12,0.481293,0.481293,0.481293,0.481293,1137,-0.013185
6,ABEV3.SA,2000-01-13,0.484589,0.484589,0.484589,0.484589,606,0.003296
7,ABEV3.SA,2000-01-14,0.494478,0.494478,0.494478,0.494478,6445,0.009889
8,ABEV3.SA,2000-01-17,0.527474,0.527474,0.527474,0.527474,25556,0.032996
9,ABEV3.SA,2000-01-18,0.527474,0.527474,0.527474,0.527474,0,0.000000


In [6]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18643 entries, 0 to 18642
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   symbol           18643 non-null  object        
 1   datetime         18643 non-null  datetime64[ns]
 2   close            18643 non-null  float64       
 3   high             18643 non-null  float64       
 4   low              18643 non-null  float64       
 5   open             18643 non-null  float64       
 6   volume           18643 non-null  int64         
 7   change_in_price  18642 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.1+ MB
